In [1]:
# 依存関係
'''
poetry add wheel

poetry add --group dev 'ipykernel' 'ipywidgets'

poetry add --group llm 'openvino'\
    "git+https://github.com/huggingface/optimum-intel.git"\
    "git+https://github.com/openvinotoolkit/nncf.git"
'''

# 使用モデル
'''
NPU:
    Phi-3-mini-4k-instruct
        INT8?FP16?
CPU:
    gemma-2b-it
        INT4
'''

'\nNPU:\n    Phi-3-mini-4k-instruct\n        INT8?FP16?\nCPU:\n    gemma-2b-it\n        INT4\n'

In [2]:
from pathlib import Path
import logging
import os
from datetime import datetime
from llm_config import LLM_MODELS_CONFIG

import openvino as ov
import nncf


INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


In [3]:
## modelがgemmaの場合は、ログインが必要
def login_huggingface_hub():
    from huggingface_hub import notebook_login, whoami
    try:
        whoami()
        print('Authorization token already provided')
    except OSError:
        notebook_login()

In [4]:
nncf.set_log_level(logging.ERROR)

core = ov.Core()

# デバイスの特定
support_devices = core.available_devices
device = 'NPU' if 'NPU' in support_devices else 'CPU'  # 実機のNPUが使えればいいのだけれど。。。

# ダウンロードするモデル
model_id = "microsoft/Phi-3-mini-4k-instruct" if device == 'NPU' else "google/gemma-2b-it"

# 一旦テストでPhi-3-mini-4k-instructを使う
# model_id = 'microsoft/Phi-3-mini-4k-instruct'

## modelがgemmaの場合は、HuggingFaceのログインが必要
if model_id == "google/gemma-2b-it":
    login_huggingface_hub()

model_name = model_id.split('/')[-1]

remote_code = LLM_MODELS_CONFIG[model_name]['remote_code']  # Phi-3はTrue

# モデルを保存するディレクトリ
model_dir = Path(f'../../model/{model_name}')
fp16_model_dir = model_dir / "FP16"  # float 16bitモデルの保存先
int8_model_dir = model_dir / "INT8"  # 量子化モデルの保存先(8bit)
int4_model_dir = model_dir / "INT4"  # 量子化モデルの保存先(4bit)

Authorization token already provided


In [5]:
# INT4の設定
compression_configs = {
    # ここのパラメータは要調整
    # "sym":          対称量子化の利用
    # 'group_size':  グループサイズ  (64, 128が無難？)
    # 'ratio':       量子化後のパラメータの割合  (0.5~0.8で試す)
    "gemma-2b-it": {
        "sym": True,
        "group_size": 64,
        "ratio": 0.6,
    },
    "default": {
        "sym": False,
        "group_size": 128,
        "ratio": 0.8,
    },
}

In [6]:
model_id

'google/gemma-2b-it'

In [7]:
# optimum-cliでモデルをopenvino形式でダウンロード
export_command_base = "optimum-cli export openvino --model {} --task text-generation-with-past".format(model_id)

def convert_to_fp16():
    global export_command_base
    export_command = ''
    # すでに存在する場合はスキップ
    if (fp16_model_dir / "openvino_model.xml" ).exists():
        return
    if remote_code:
        # Phi-3のみ
        export_command_base += " --trust-remote-code"
    export_command = export_command_base + " --weight-format fp16"
    export_command += " " + str(fp16_model_dir)
    # モデルのダウンロード開始時間
    start_model_download = datetime.now()
    print('export_command:', export_command)
    os.system(export_command)  # モデルのダウンロード
    # モデルのダウンロード終了時間
    end_model_download = datetime.now() - start_model_download
    print('export done', end_model_download.total_seconds())


def convert_to_int8():
    global export_command_base
    if (int8_model_dir / "openvino_model.xml").exists():
        return
    if remote_code:
        export_command_base += " --trust-remote-code"
    export_command = export_command_base + " --weight-format int8"
    export_command += " " + str(int8_model_dir)
    # モデルのダウンロード開始時間
    start_model_download = datetime.now()
    print('export_command:', export_command)
    os.system(export_command)  # モデルのダウンロード
    # モデルのダウンロード終了時間
    end_model_download = datetime.now() - start_model_download
    print('export done', end_model_download.total_seconds())

def convert_to_int4():
    global export_command_base
    if (int4_model_dir / "openvino_model.xml").exists():
        return
    if remote_code:
        export_command_base += " --trust-remote-code"
    # 量子化の設定
    model_compression_params  = compression_configs.get(model_name, compression_configs["default"])
    export_command = export_command_base + " --weight-format int4"
    int4_compression_args = " --group-size {} --ratio {}".format(model_compression_params["group_size"], model_compression_params["ratio"])
    if model_compression_params["sym"]:
        int4_compression_args += " --sym"
    export_command += int4_compression_args + " " + str(int4_model_dir)
    # モデルのダウンロード開始時間
    start_model_download = datetime.now()
    print('export_command:', export_command)
    os.system(export_command)  # モデルのダウンロード
    # モデルのダウンロード終了時間
    end_model_download = datetime.now() - start_model_download
    print('export done', end_model_download.total_seconds())


In [10]:
# deviceがNPUならINT8, それ以外はINT4
convert_to_int8() if device == "NPU" else convert_to_int4()
# convert_to_fp16()

export_command: optimum-cli export openvino --model google/gemma-2b-it --task text-generation-with-past --weight-format int4 --group-size 64 --ratio 0.6 --sym ../../model/gemma-2b-it/INT4
export done 0.026528


In [11]:
# モデルが保存されているディレクトリのサイズを確認

fp16_weights = fp16_model_dir / "openvino_model.bin"
int8_weights = int8_model_dir / "openvino_model.bin"
# int4_weights = int4_model_dir / "openvino_model.bin"

if fp16_weights.exists():
    print(f"Size of FP16 model is {fp16_weights.stat().st_size / 1024 / 1024:.2f} MB")

# ファイルのサイズを確認
# for precision, compressed_weights in zip([8, 4], [int8_weights, int4_weights]):
for precision, compressed_weights in zip([8], [int8_weights]):
    if compressed_weights.exists():
        print(f"Size of model with INT{precision} compressed weights is {compressed_weights.stat().st_size / 1024 / 1024:.2f} MB")
    if compressed_weights.exists() and fp16_weights.exists():
        print(f"Compression rate for INT{precision} model: {fp16_weights.stat().st_size / compressed_weights.stat().st_size:.3f}")

In [35]:
device

'CPU'

In [36]:
from ipywidgets import widgets

available_models = []
# if int4_model_dir.exists():
#     available_models.append("INT4")
if int8_model_dir.exists():
    available_models.append("INT8")
if fp16_model_dir.exists():
    available_models.append("FP16")

model_to_run = widgets.Dropdown(
    options=available_models,
    value=available_models[0],
    description="Model to run:",
    disabled=False,
)

model_to_run

Dropdown(description='Model to run:', options=('INT8', 'FP16'), value='INT8')

In [37]:
model_to_run.value

'INT8'

In [38]:
from transformers import AutoConfig, AutoTokenizer
from optimum.intel.openvino import OVModelForCausalLM

# if model_to_run.value == "INT4":  # 4bitモデルを使う場合
#     model_dir = int4_model_dir
if model_to_run.value == "INT8":  # 8bitモデルを使う場合
    model_dir = int8_model_dir
else:
    model_dir = fp16_model_dir  # 16bitモデルを使う場合
print(f"Loading model from {model_dir}")

ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

tok = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

ov_model = OVModelForCausalLM.from_pretrained(
    model_dir,
    device=device,
    ov_config=ov_config,
    config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True),
    trust_remote_code=True,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading model from ../../model/Phi-3-mini-4k-instruct/INT8


/home/korryu/Documents/TECH.C/Taltner/server/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The argument `trust_remote_code` is to be used along with export=True. It will be ignored.
Compiling the model to CPU ...


In [67]:
# system prompt
SYSTEM_PROMPT = '''\
# Instructions
You are an assistant designed to help users facilitate conversations with people they are meeting for the first time.
You will assist the user by smoothing the conversation between them and the other party.
Consider the following elements and present **four topics in JSON format**.

# Conditions
1. Provide topics suitable for conversation with a person you are meeting for the first time.
2. Consider the flow of the conversation when presenting the next topic.
3. Present topics in {{short content at the word level}}.
4. Output language should be {{Japanese}}.

**Present in the following format**
```json
{
  "topics": [
    "Topic 1",
    "Topic 2",
    "Topic 3",
    "Topic 4"
  ]
}
```

# Example 1
## Input
自分「最近ハイキングにハマってるんだけど、君もアウトドア活動は好き？」
相手「うん、たまには山に登ったりするよ。」

## Output
```json
{
  "topics": [
    "おすすめのハイキングスポット",
    "必要な装備",
    "最近の登山経験",
    "季節ごとの活動"
  ]
}
```

# Example 2
## Input
自分「サスペンス映画が好きでよく見るんだけど、好きなジャンルは？」
相手「僕はドキュメンタリーが好きだね。」

## Output
```json
{
  "topics": [
    "おすすめのドキュメンタリー",
    "サスペンス映画の魅力",
    "映画鑑賞の場所",
    "映画の感想交換"
  ]
}
```

# Example 3
## Input
自分「テクノロジーのニュースを追ってるんだけど、興味はある？」
相手「うん、特にAIに関しては詳しく知りたい。」

## Output
```json
{
  "topics": [
    "最新のAI技術",
    "AIの社会的影響",
    "読んでるニュースのサイト",
    "AIに関するおすすめの本"
  ]
}
```

# Example 4
## Input
自分「コーヒーを淹れるのが趣味なんだけど、君はコーヒーは好きかな？」
相手「大好きだよ。いつもカフェで新しい豆を試してる。」

## Output
```json
{
  "topics": [
    "おすすめのコーヒー豆",
    "カフェのおすすめ",
    "自宅でのコーヒーの淹れ方",
    "コーヒーと健康"
  ]
}
```

# Example 5
## Input
自分「SNSで美術館の展示を見るのが好きなんだけど、美術館は好き？」
相手「美術館はいいよね、特に近代美術が好き。」

## Output
```json
{
  "topics": [
    "おすすめの近代美術館",
    "最近訪れた展示",
    "アートコレクション",
    "美術館のおすすめ展示"
  ]
}
```

# Example 6
## Input
自分「自分でミュージックを作るのが趣味なんだけど、君も音楽は好き？」
相手「音楽は大好きで、よくライブに行くよ。」

## Output
```json
{
  "topics": [
    "おすすめのアーティスト",
    "最近のライブ体験",
    "音楽制作のヒント",
    "音楽ジャンルの好み"
  ]
}
```

# Example 7
## Input
自分「料理のYouTubeチャンネルを見るのが好きなんだけど、君はどんなチャンネルを見る？」
相手「テクノロジーレビューのチャンネルをよく見てるよ。」

## Output
```json
{
  "topics": [
    "おすすめのテクノロジーチャンネル",
    "テクノロジーチャンネルを見る理由",
    "YouTubeで学べること",
    "最新のテクノロジートレンド"
  ]
}
```

# Example 8
## Input
自分「海外旅行が趣味で、最近はアジアを中心に回ってるんだ。君も旅行は好き？」
相手「うん、特にヨーロッパが好きで、よく行くよ。」

## Output
```json
{
  "topics": [
    "おすすめのヨーロッパの国",
    "ヨーロッパ旅行の魅力",
    "旅行の計画方法",
    "旅行でのエピソード"
  ]
}
```

# Example 9
## Input
自分「仕事でプログラミングをしてるんだけど、君もIT関連の仕事をしてる？」
相手「はい、システムエンジニアとして働いています。」

## Output
```json
{
  "topics": [
    "プログラミング言語の選択",
    "プロジェクトの課題",
    "IT業界のトレンド",
    "キャリアパスの話"
  ]
}
```

# Example 10
## Input
自分「ブログを書いていて、主に旅行についての情報を発信してるんだ。君も何か書いてる？」
相手「写真についてのブログをたまに更新してるよ。」

## Output
```json
{
  "topics": [
    "ブログのコンテンツアイデア",
    "写真技術のポイント",
    "旅行の記録",
    "SNSでの情報共有"
  ]
}
```
'''

In [68]:
# モデルの動作確認
tokenizer_kwargs = LLM_MODELS_CONFIG[model_name].get("tokenizer_kwargs", {})

conversation = '''\
自分「機械学習の授業俺も興味あるんだけど難易度どんな感じ？」
相手「結構難しいと思うけど、興味あるなら楽しめると思うよ」
'''

start_message: str = LLM_MODELS_CONFIG[model_name]['start_message']
prompt_template: str = LLM_MODELS_CONFIG[model_name]['prompt_template']

sys_prompt = start_message.format(
    SYSTEM_PROMPT=SYSTEM_PROMPT
)

prompt = sys_prompt + prompt_template.format(
    user=conversation
)

input_tokens = tok(prompt, return_tensors="pt", **tokenizer_kwargs)

In [88]:
%%time

answer = ov_model.generate(
    **input_tokens,
    max_new_tokens=110,
    temperature=0.7,
    do_sample=True
)

decoded_answer = tok.batch_decode(answer, skip_special_tokens=True)[0]
print(decoded_answer)

# Instructions
You are an assistant designed to help users facilitate conversations with people they are meeting for the first time.
You will assist the user by smoothing the conversation between them and the other party.
Consider the following elements and present **four topics in JSON format**.

# Conditions
1. Provide topics suitable for conversation with a person you are meeting for the first time.
2. Consider the flow of the conversation when presenting the next topic.
3. Present topics in {{short content at the word level}}.
4. Output language should be {{Japanese}}.

**Present in the following format**
```json
{
  "topics": [
    "Topic 1",
    "Topic 2",
    "Topic 3",
    "Topic 4"
  ]
}
```

# Example 1
## Input
自分「最近ハイキングにハマってるんだけど、君もアウトドア活動は好き？」
相手「うん、たまには山に登ったりするよ。」

## Output
```json
{
  "topics": [
    "おすすめのハイキングスポット",
    "必要な装備",
    "最近の登山経験",
    "季節ごとの活動"
  ]
}
```

# Example 2
## Input
自分「サスペンス映画が好きでよく見るんだけど、好きなジャンルは？」
相手「僕はドキュメンタリーが好きだね。」

## Output
```json
{
  "

In [149]:
# Aiが生成したテキスト部分のみを取得
def extract_generated_text(decoded_answer: str) -> str:
    generated_text = []
    capturing = False
    append_closure = False
    closure_text = ""

    lines = decoded_answer.strip().split('\n')[::-1]

    # コードブロックが正常に終了しない場合に使用
    def extract_json_text(lines):
        generated_text = []
        for line in lines:
            if "```json" in line:
                return generated_text
            generated_text.append(line)


    # テキストを行ごとに分割して逆順で処理
    for line in lines:
        stripped_line = line.strip()

        # JSONでコードブロックの終わりまで正常に出力される場合　　
        # コードブロックの終わりを見つけた場合
        if line.strip() == "```":
            capturing = True
            continue  # 次の行から取得開始

        # コードブロックの始まりを見つけたら終了
        if capturing and "```json" in stripped_line:
            print('正常に終了')
            break

        if capturing:
            generated_text.append(line)

        #　コードブロックの終わりまで正常に出力されない場合、特定の状態で処理
        if (not capturing) and ("```json" in stripped_line):
            end_line = lines[0]
            # 最後の行が'}'なら、そこと、次の'```json'の間を取得
            if '}' in end_line:
                print('最後の行が"}"')
                append_closure = False
                generated_text = extract_json_text(lines)
            # 最後の行が']'なら、そこと、次の'```json'の間を取得し、最後に'\n}'を追加
            elif "]" in end_line:
                print('最後の行が"]"')
                append_closure = True
                closure_text = "\n}"
                generated_text = extract_json_text(lines)
            # それ以外は、次の'```json'の間を取得し、最後が'"'なら'\n]\n}'を追加、それ以外は'"\n]\n}'を追加
            else:
                print('最後の行がそれ以外')
                generated_text = extract_json_text(lines)
                if end_line.strip().endswith("\""):
                    append_closure = True
                    closure_text = '\n  ]\n}'
                else:
                    append_closure = True
                    closure_text = '\"\n  ]\n}'
            break

    reversed_text = "\n".join(reversed(generated_text))

    if append_closure:
        reversed_text += closure_text

    return reversed_text

In [152]:
decoded_answer_example = '''
## Output
```json
{
  "topics": [
    "ブログのコンテンツアイデア",
    "写真技術のポイント",
    "旅行の記録",
    "SNSでの情報共有"
  ]
}
```
 自分「機械学習の授業俺も興味あるんだけど難易度どんな感じ？」
相手「結構難しいと思うけど、興味あるなら楽しめると思うよ」 ```json
{
  "topics": [
    "基本的な機械学習技術",
    "困難な問題解決の戦略",
    "実用的な機械学習の事例",
    "授業の質の向上方法"
  ]
}
```
'''

# 最後が```jsonで終わってしまう場合、正常に終了しないから、そこを明日直す

# 実際の関数を使用して結果を得る
result = extract_generated_text(decoded_answer_example)
print(result)

最後の行がそれ以外
"
  ]
}
